In [1]:
!pip install gdown
import gdown

# URL to the Google Drive file
drive_link = 'https://drive.google.com/uc?id=1nYSJSXOZhywPzWHbq3csuWvTP6UPc7DR'
# Download the file
output_path = './'
gdown.download(drive_link, output_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1nYSJSXOZhywPzWHbq3csuWvTP6UPc7DR
To: /content/BCCD.v4-416x416_aug.yolov5pytorch.zip
100%|██████████| 13.3M/13.3M [00:00<00:00, 161MB/s]


'./BCCD.v4-416x416_aug.yolov5pytorch.zip'

In [2]:
import zipfile
import os

def unzip_folder(zip_path, extract_to):
    """
    Unzips the specified zip file into the specified directory.

    :param zip_path: Path to the zip file
    :param extract_to: Directory to extract the contents to
    """
    # Ensure the extract_to directory exists
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    # Open the zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all contents
        zip_ref.extractall(extract_to)
        print(f"Extracted all files to {extract_to}")

# Example usage
zip_path = '/content/BCCD.v4-416x416_aug.yolov5pytorch.zip'
extract_to = './'
unzip_folder(zip_path, extract_to)

Extracted all files to ./


In [3]:
# import os,shutil

# cPath=os.getcwd()
# inputPar=os.path.join(cPath,"data")
# outPar=os.path.join(cPath,"Dataset")

# def copyFiles(fldr,files,category,inputChild):

#     outSub=os.path.join(outPar,category,fldr)
#     os.makedirs(outSub,exist_ok=True)

#     for fl in files:
#         finput=os.path.join(inputChild,fl)
#         fout=os.path.join(outSub,fl)
#         shutil.copy(finput,fout)

# for folder in os.listdir(inputPar):
#     inputChild=os.path.join(inputPar,folder)
#     files=os.listdir(inputChild)

#     trainLen=int(len(files)*0.7)
#     valLen=int(len(files)*0.2)
#     testLen=int(len(files)*0.1)
#     trainset=files[:trainLen]
#     valSet=files[trainLen:trainLen+valLen]
#     testSet=files[trainLen+valLen:]

#     copyFiles(folder,trainset,"train",inputChild)
#     copyFiles(folder,valSet,"valid",inputChild)
#     copyFiles(folder,testSet,"test",inputChild)

In [4]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor
import time

startTime=time.time()
def copy_files(src_dir, dst_dir):
    """Copy files from src_dir to dst_dir."""
    os.makedirs(dst_dir, exist_ok=True)
    for file in os.listdir(src_dir):
        shutil.copy(os.path.join(src_dir, file), os.path.join(dst_dir, file))

def process_folder(inputPar, outPar, folder):
    """Process 'images' and 'labels' subfolders within the given folder."""

    if folder in ["train", "valid", "test"]:
      inputChild = os.path.join(inputPar, folder)

      for subfldr in ["images", "labels"]:
          inputSubChild = os.path.join(inputChild, subfldr)
          outChild = os.path.join(outPar, subfldr, folder)

          if os.path.exists(inputSubChild):
              copy_files(inputSubChild, outChild)

def main():
    cPath = os.getcwd()
    inputPar = cPath
    outPar = os.path.join(cPath, "Dataset")

    folders = ["train", "valid", "test"]

    with ThreadPoolExecutor() as executor:
        executor.map(lambda folder: process_folder(inputPar, outPar, folder), folders)

if __name__ == "__main__":
    main()

endTime=time.time()

print("process Completed in !!",endTime-startTime)

process Completed in !! 0.2411813735961914


In [17]:
import yaml

# Define the data configuration
data_config = {
    'train': '/content/Dataset/images/train',
    'val': '/content/Dataset/images/valid',
    'nc': 3,  # number of classes
    'names': ['Platelets', 'RBC', 'WBC']
}

# Write the configuration to a YAML file
with open('custom_dataset.yaml', 'w') as file:
    yaml.dump(data_config, file)

In [6]:
import ast

with open(r"/content/custom_dataset.yaml") as f:
    dataYaml=f.readlines()

num_cls=0
for line in dataYaml:

    if line.startswith("nc:"):
        num_classes=int(line.split("nc:")[-1])
        num_cls=num_classes
        break

with open("classes.txt","w") as f:

    for cls in dataYaml[1:num_cls+1]:
        f.write(cls.lstrip("- "))

In [7]:
with open("classes.txt") as f:
  classes=f.readlines()

classes=[i.rstrip() for i in classes]
print(classes)

['Platelets', 'RBC', 'WBC']


In [8]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [9]:
# !pip install -r requirements.txt


In [19]:
!python train.py --img 640 --batch 16 --epochs 25 --data "/content/yolov5/custom_dataset.yaml" --weights yolov5s.pt --cache

2024-07-17 04:48:48.995195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 04:48:48.995250: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 04:48:48.997186: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/yolov5/custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=25, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False

In [11]:
# #------------------ pretrained model ---------------------
# drive_link = 'https://drive.google.com/uc?id=1PHlMNPKfN6WIChYmzJGjMGRez_RQaXeh'
# # Download the file
# output_path = './'
# gdown.download(drive_link, output_path, quiet=False)

In [12]:
# !yolo task=detect mode=train model=best.pt data=/content/data.yaml epochs=5 plots=True

In [27]:
import torch
import cv2
import numpy as np
from pathlib import Path

# Load the YOLOv5 model
model_path = '/content/yolov5/runs/train/exp4/weights/best.pt'  # Replace with your model path
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

# Load an image
img_path = '/content/Dataset/images/test/BloodImage_00337_jpg.rf.b6cb228440b9158cafec01a0351e3aad.jpg'  # Replace with your image path
img = cv2.imread(img_path)

# Convert the image to RGB (YOLOv5 expects RGB images)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Perform inference
results = model(img_rgb)

# Print results
results.print()  # Print results to the console

# Extract bounding boxes and labels
bbox_tensor = results.xyxy[0].cpu()  # Move the tensor to CPU
bbox_array = bbox_tensor.numpy()  # Convert to NumPy array

for bbox in bbox_array:
    x1, y1, x2, y2, conf, cls = bbox
    label = model.names[int(cls)]
    conf = f'{conf:.2f}'

    # Draw bounding box
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    cv2.putText(img, f'{label} {conf}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Save the output image
output_path = '/content/Untitled Folder/output_image.jpg'  # Replace with your desired output path
cv2.imwrite(output_path, img)




Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
image 1/1: 416x416 29 RBCs, 1 WBC
Speed: 2.2ms pre-process, 7.0ms inference, 6.0ms NMS per image at shape (1, 3, 640, 640)


True

In [36]:
import torch
import cv2
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt

# Define our imshow function
def imshow(title = "Image", image = None, size = 10):
    w, h = image.shape[0], image.shape[1]
    aspect_ratio = w/h
    plt.figure(figsize=(size * aspect_ratio,size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

# Load the YOLOv5 model
model_path = '/content/yolov5/runs/train/exp4/weights/best.pt'  # Replace with your model path
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

# Load an image
img_path = '/content/Dataset/images/test/BloodImage_00090_jpg.rf.cdbf8f6ed3b93fa902a0bc991132cb40.jpg'  # Replace with your image path
img = cv2.imread(img_path)

# Convert the image to RGB (YOLOv5 expects RGB images)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Perform inference
results = model(img_rgb)

# Print results
results.print()  # Print results to the console

# Extract bounding boxes and labels
bbox_tensor = results.xyxy[0].cpu()  # Move the tensor to CPU
bbox_array = bbox_tensor.numpy()  # Convert to NumPy array

# Non-Max Suppression parameters (if you want to customize)
iou_threshold = 0.5  # Intersection over Union threshold
confidence_threshold = 0.25  # Confidence score threshold

# Apply NMS
filtered_boxes = []
for bbox in bbox_array:
    if bbox[4] >= confidence_threshold:
        filtered_boxes.append(bbox)

filtered_boxes = np.array(filtered_boxes)

# Convert to the format required by cv2.dnn.NMSBoxes
boxes = [[int(box[0]), int(box[1]), int(box[2] - box[0]), int(box[3] - box[1])] for box in filtered_boxes]
scores = [box[4] for box in filtered_boxes]

# Apply NMS
indices = cv2.dnn.NMSBoxes(
    bboxes=boxes,
    scores=scores,
    score_threshold=confidence_threshold,
    nms_threshold=iou_threshold
)

# Draw bounding boxes and labels
for i in indices:

    x1, y1, w, h = boxes[i]
    x2, y2 = x1 + w, y1 + h
    conf, cls = filtered_boxes[i][4], filtered_boxes[i][5]
    label = model.names[int(cls)]
    conf = f'{conf:.2f}'

    # Draw bounding box
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, f'{label} {conf}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


# Save the output image
output_path = '/content/Untitled Folder/output_image_2.jpg'  # Replace with your desired output path
cv2.imwrite(output_path, img)
imshow("image",img)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
image 1/1: 416x416 3 Plateletss, 26 RBCs, 1 WBC
Speed: 2.1ms pre-process, 6.5ms inference, 6.5ms NMS per image at shape (1, 3, 640, 640)


yes
